## Notes on ice thickness estimation from airborne LiDAR
Adam Steer, January 2016.
adam.d.steer@gmail.com

For this project, we have a bunch of LIDAR points georeferenced relative to an ITRF08 ellipsoid. What we need is the height of each point relative to sea level.

But where exactly is the sea surface? We can estimate it from a gravity model, plus a tide model, plus some knowledge of dynamic sea surface topography (wind pressure etc. ).

Or we can use a sensor-based approach, since we know some points are returns from water, or extremely thin new ice.

This document describes the second approach.

In [ ]:
#setup
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema
from sklearn import neighbors as nb
from shapely import geometry
from datetime import datetime

In [1]:
#setting up a file path for IO
infile_dir = '/media/adam/data/is6_f11/lidar.matlab/swaths/'
infile = 'is6_f11_pass1_aa_nr2_522816_523019_c.xyz'
outfilename = 'is6_f11_pass1_aa_nr2_522816_523019_zi_data.xyz'

In [2]:
#these parameters drive the sea ice thickness estimation model
d_snow = 305
sd_dsnow = 10
d_ice = 850
sd_dice = 10
d_water = 1028
sd_dwater = 1

## Function declarations

In [4]:
# this function is the snow depth model. Parameters come from Steer et al (2016):
#Estimating snow depth from altimery for East Antarctic pack ice

def compute_zs(tf, s_i, tf_uncert):
    """
    Take in the total freeboard (tf, float array), slope and intercept for an empirical model
    of snow depth from elevation (s_i, tuple) and the total freeboard uncertainty
    (tf_uncert, float array)

    Return a snow depth, with an associated uncertainty.
    zs, uncert = compute_zs(tf, ([slope, intercept]), tf_uncert)
    """
    
    zs = (params[0] * tf) + s_i[1]

    zs_uncert = 0.72 * tf_uncert

    return zs, zs_uncert


In [5]:
# next define the model for estimating ice thickness from total freeboard, snow depth
# and some density parameters:

def compute_zi(tf, zs, d_ice, d_water, d_snow, sd_tf, sd_zs, sd_dsnow, \
               sd_dice, sd_dwater):
    """"
    sea ice thickness from elevation, and propagation of uncertainties
    after Kwok, 2010; kwok and cunningham, 2008
    equations:
    4: sea ice thickness from elevation
    6: taylor series expansion of variance/covariance propagation using
    partial derivatives
    8, 9 and 10: partial derivatives used in the taylor series
    """
    zi = (d_water / (d_water-d_ice)) * tf - ((d_water-d_snow) / \
         (d_water - d_ice)) * zs

    zi_uncert = sd_tf**2 * (d_water / (d_water - d_ice))**2 + \
           sd_zs**2 * ((d_snow - d_water) / (d_water - d_ice))**2 + \
           sd_dsnow**2 * (zs / (d_water - d_ice))**2 + \
           sd_dice**2 * (tf /  (d_water - d_ice))**2 + \
           sd_dwater**2 * (((-d_ice * tf) + ((d_ice-d_snow) * zs)) / \
           (d_water - d_ice)**2)**2

    return zi, zi_uncert


In [6]:
# next a utility function to make zero mean data
def zero_mean(data):
    """
    take in any array of data
    retuun an array modified such that mean(data) == 0
    """

    oldmean = np.mean(data)
    return data - oldmean

In [7]:
# next, the first function for finding sea level keypoints
def find_lowpoints(elev_data, nhood):
    """
    Pass in an array of elevation vlues and an integer  number of points to use
    as a neighbourhood.
    
    Get back the indices of local miminum elevation values for the chosen
    neighbourhoods as a tuple.
    
    http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/
    scipy.signal.argrelextrema.html#scipy.signal.argrelextrema
    """
    le_idx = argrelextrema(np.array(elev_data), np.less, order=nhood)
    return list(le_idx[0])

In [8]:
#part two of fiding water - intensity percentile based low pass filter
def find_low_intens(intens,pntile):
    """
    Take in some return intensity data (array) and a percentile (float)
    Return the indices of chosen percentil of intensity values
    """
    
    li_idx =  np.where(intens <= np.percentile(intens, pntile))
    return list(li_idx[0])

In [9]:
# gluing it together, using the three functions above

def find_water_keys(xyzi, e_p, i_p):
    """
    Pass in:
    - a set of X,Y,Z and Intensity points
    - the number of points to use as a neighbourhood for choosing low elevation
    - a percentile level for intensity thresholding
    Get back:
    - a set of XYZI points corresponding to 'local sea level'
    """
    #find lowest intensity values
    low_intens_inds = find_low_intens(xyzi[:, 3], i_p)
    li_xyzi = xyzi[low_intens_inds, :]

    #find low elevations
    low_elev_inds = find_lowpoints(li_xyzi[:, 2], e_p)
    low_xyzi = li_xyzi[low_elev_inds, :]
    #low_xyzi = np.squeeze(low_xyzi)

    return low_xyzi

In [10]:
# moving on to some filtering! Point cloud smoothing with respect to 3D space

In [11]:
def query_tree(pointcloud, tree, radius):
    nhoods = tree.query_radius(pointcloud[:,0:3], r=radius)
    #new_z = np.median(xyzi[nhoods[0],2])

    new_z = []
    i = 0
    for nhood in nhoods:
        #print(nhood)
        new_z.append(np.nanmean(pointcloud[nhood[:],2]))

        i += 1
    return new_z

def spatialmedian(pointcloud,radius):
    """
    Using a KDTree and scikit-learn's query_radius method,
    ingest an n-d point cloud with the XYZ coordinates occupying
    the first three columns respectively (pointcloud[:,0:3]) and a radius in
    metres (or whatever units the pointcloud uses).
    Returns a new set of Z values which contain the median Z value of points within
    radius r of each point.
    """

    from sklearn import neighbors as nb

    from datetime import datetime
    startTime = datetime.now()

    print(startTime)

    tree = nb.KDTree(pointcloud[:,0:3], leaf_size=60)

    print(datetime.now() - startTime)

    nhoods = tree.query_radius(pointcloud[:,0:3], r=radius)
    #new_z = np.median(xyzi[nhoods[0],2])

    new_z = []
    i = 0
    for nhood in nhoods:
        #print(nhood)
        new_z.append(np.median(pointcloud[nhood[:],2]))
        #print(pointcloud[i,:])
        #print(new_z[i])

        i += 1

    print(datetime.now() - startTime)

    # we really want to return the tree - only generate it once per operation!
    return new_z, tree

## Applying it all! Setting up reference points and levelling the data

In [ ]:
#load up a point cloud
lidar = np.genfromtxt(infile_dir + infile)

input_points = lidar[(lidar[:,0] >= -150) & (lidar[:,0] <= 130) & 
                     (lidar[:,1] >= -30) & (lidar[:,1] <= 275) ]

In [ ]:

#pick out XYZ and intensity
xyzi = input_points[:, 1:5]

#find the lowest elevation point
lowestpoint = np.min(input_points[:, 3])

#find the sd of intensity
sd_intens = np.std(input_points[:, 4])

#compute a reference point set
xyz_fitpoints = np.squeeze(find_water_keys(xyzi, 100, 1.2))

#using a KNeighbours regression, fit a surface to the reference points
# just found. Here we declare a KNR object
knnf = nb.KNeighborsRegressor(10, algorithm='kd_tree', n_jobs=-1)

# here's the fitting part
knnf.fit(np.array([xyz_fitpoints[:, 0], xyz_fitpoints[:, 1]]).reshape(len(xyz_fitpoints[:, 1]), 2), xyz_fitpoints[:, 2])

#now - predict Z for the entire pointcloud
z_fit = knnf.predict(np.array([xyzi[:,0], xyzi[:, 1]]).reshape(len(xyzi[:, 0]), 2))

#now, use z_fit to 'level' the data
xyzi_mod = np.column_stack([input_points[:,1], input_points[:,2], \
                            input_points[:,3] - z_fit, input_points[:,4]])

#next, apply a spatial filter. It is actually the mean of each neighborhood at present
median_z, points_kdtree = spatialmedian(xyzi_mod, 2)

# OK, now we build a new cloud with median Z
xyzi2 = np.column_stack([input_points[:,1], input_points[:,2], \
                            median_z, input_points[:,4]])


## Part 2: making sea ice thickness!

In [ ]:

#still unsure why it isn't an no.array already...
# assigning the median filtered elevations as total freeboard (tf)
tf = np.array(median_z)

#this is cheating, but we don't want positive draft!
# I'll track these points... 
tf[np.where(tf < 0)] = 0

#get total freeboard uncertainty
sd_tf = input_points[:, 8]

#compute snow depth based on an empirical model
s_i = ([0.72, 0.001])
zs, zs_uncert = compute_zs(tf, s_i, sd_tf)

#...and finally ice thickness!
zi, zi_uncert = compute_zi(tf, zs, d_ice, d_water, d_snow, sd_tf, \
                           zs_uncert, sd_dsnow, sd_dice, sd_dwater)

np.mean(zs)
np.std(zs)

np.mean(zi)
np.median(zi)
np.std(zi)

## To do:

- make a pointcloud class
- turn the functions above into methods and functions on the class
- for example (a few methods below):
    
    - pc = pointcloud()
    - pc.medianfilter(args)
    - pc.tolas(args)
    - pc.tonetcdf(args)
    - pc.anglefilter(args)
    - pc.rangegate(args)
    - pc.density(args)
    - pc.grid(args)
    - pc.normals()
    - pc.surf() (maybe)
    - pc.outliers()
    
- and some sea ice specific ones:
    - pc.estimatezs(modelparams)
    - pc.estimatezi(modelparams)
    
- and really later:
    - pc.make(range, angle, trajectory, transforms)
    - pc.computeUncert(args)

All these functions exist, and can be called in various ways - but wrapping them up in a class would be nice and neat.